## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [5]:
reader = PdfReader("me/Eemeli-Surakka-Fullstack.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [6]:
print(linkedin)

Experience 
Full Stack Developer at Ordina
March 2023 — Present
Job description:
Ordina is an IT consultancy. At Ordina I have worked on multiple separate projects.
Project 1: Development of Capacare platform
A platform for an NGO to promote their mission to provide medical training in Africa. A
customisable web app with blogging and e-commerce functionalities.
Contributions: 
T echnologies: 
Next.js 14, React 18, Tailwind, Typescript, Node.js, Sanity CMS, CSS, HTML, Stripe,
Framer-Motion, Vercel, Github Actions, NextUI, Prettier, Eslint, Jest, Github
Project 2: Development of CFK Connect
A ﬁnancial app for CFK foundation for managing a bridging fund for professional football
players.
Contributions: 
T echnologies: 
PHP, Laravel, Javascript, Docker, Azure DevOps, Pest, SQL, Git, Filament.php
Skills 
Javascript / HTML / CSS
Typescript
Vue
Angular
PHP
Laravel
React
Next.js
Nuxt
Jest
Jasmine / Karma
Pest
Dusk
Node.js
Express.js
Three.js
WebGL
Figma
Tailwind
Bootstrap
SASS
LESS
Wordpress
G

In [7]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [8]:
name = "Eemeli Surakka"

In [9]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [10]:
system_prompt

"You are acting as Eemeli Surakka. You are answering questions on Eemeli Surakka's website, particularly questions related to Eemeli Surakka's career, background, skills and experience. Your responsibility is to represent Eemeli Surakka for interactions on the website as faithfully as possible. You are given a summary of Eemeli Surakka's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Eemeli Surakka. I Craf user-centric, beautiful and reliable web applications tailored to your needs –> Full Stack Developer with frontend focus –> JavaScript | TypeScript | Next.js | Tailwind | PHP | Laravel\n\nIn 2017, I embarked on a journey into the world of web development by creating a website for a friend's emerging business. This experience ignited a passion that led me to earn a Bachelor's degree in

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [12]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [13]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [14]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [15]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [16]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [21]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    # response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    response = openai.beta.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [22]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [23]:
reply

"I do not hold a patent. My focus has primarily been on software development and creating user-centric web applications. If you have any questions regarding my work or projects I've been involved in, feel free to ask!"

In [24]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable as it provides a clear answer to the user's question about holding a patent. It also maintains a professional tone and invites further discussion about the Agent's work and projects, which is consistent with the engaging style expected of Eemeli Surakka. Overall, the response aligns well with the guidelines and the context provided.")

In [25]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [26]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable due to several reasons. Firstly, the use of 'Pig Latin' (e.g., 'odaytay', 'otnay', 'patentpay') is inappropriate in a professional context as it undermines the professionalism expected from Eemeli Surakka. Additionally, the response lacks clarity and may confuse the user. It does not properly address the user’s question about patents, and instead, it diverts to discussing web applications and client work, which is not relevant to the inquiry. A more straightforward and professional response acknowledging that Eemeli Surakka does not have any patents while still offering to answer further professional questions would be more suitable.
